<a href="https://colab.research.google.com/github/sokanaid/CourseProject2022BiologicalMath/blob/main/model_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучение моделей предсказания числинности на плато

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
np.random.seed(0)

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/sim_tables

Mounted at /content/drive/
/content/drive/MyDrive/sim_tables


## Подготовка датасета

In [16]:
file_name = "/content/drive/MyDrive/sim_tables/data_set.csv"
sim_frame = pd.read_csv(file_name, sep='\t').drop("Unnamed: 0", axis = 1)

In [17]:
sim_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290525 entries, 0 to 290524
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   b              290525 non-null  float64
 1   d              290525 non-null  float64
 2   death_r        290525 non-null  float64
 3   dd             290525 non-null  float64
 4   sd_b           290525 non-null  float64
 5   sd_d           290525 non-null  float64
 6   area_length_x  290525 non-null  float64
 7   initial_pop    290525 non-null  float64
 8   plateau_pop    290525 non-null  float64
dtypes: float64(9)
memory usage: 19.9 MB


In [18]:
print("total count of simulations:", len(sim_frame))
df = sim_frame[sim_frame["area_length_x"]==100.0].drop("area_length_x", axis = 1)
df1 = df.drop_duplicates(subset=['b', 'd', 'death_r', 'dd', 'sd_b', 'sd_d', 'initial_pop']).reset_index(drop=True)
print("total count of simulations to use in learning:", len(df))

total count of simulations: 290525
total count of simulations to use in learning: 221711


In [19]:
# рапределение размеров в изначальном датасете
sim_frame["area_length_x"].value_counts()

100.0     221711
1100.0      3127
3100.0      2345
2100.0      2345
4100.0      2345
2500.0      2343
3300.0      2343
3500.0      2343
2300.0      2343
300.0       2343
1700.0      2343
3900.0      2343
2900.0      2343
4300.0      2343
4500.0      2343
500.0       2343
1900.0      2343
3700.0      2343
5100.0      2343
2700.0      2343
700.0       2343
900.0       2343
4700.0      2343
1300.0      2343
1500.0      2343
5700.0      2342
5500.0      2342
4900.0      2342
5900.0      2342
5300.0      2342
2600.0        14
1600.0        14
3600.0        14
600.0         14
4600.0        13
5600.0        13
Name: area_length_x, dtype: int64

In [20]:
# распределение параметров
for column in list(df.columns.values):
  print(column)
  print(df[column].describe())

b
count    221711.000000
mean          0.674166
std           0.228180
min           0.100000
25%           0.550000
50%           0.750000
75%           0.850000
max           0.950000
Name: b, dtype: float64
d
count    221711.000000
mean          0.362323
std           0.221028
min           0.100000
25%           0.200000
50%           0.300000
75%           0.500000
max           0.900000
Name: d, dtype: float64
death_r
count    221711.000000
mean          5.360903
std           2.961302
min           0.100000
25%           3.000000
50%           5.000000
75%           8.000000
max          10.000000
Name: death_r, dtype: float64
dd
count    221711.000000
mean          0.549894
std           0.287142
min           0.100000
25%           0.300000
50%           0.550000
75%           0.800000
max           1.000000
Name: dd, dtype: float64
sd_b
count    221711.000000
mean          0.725233
std           0.167625
min           0.500000
25%           0.600000
50%           0.750000
75%

In [21]:
X = df.drop("plateau_pop", axis =1)
y = df["plateau_pop"]

In [22]:
X.head()

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,0.35,0.1,3.0,0.9,0.50,0.65,100.0
1,0.45,0.3,7.0,0.7,0.95,0.80,1.0
2,0.95,0.8,9.0,0.2,0.80,0.95,1.0
3,0.35,0.2,1.0,0.6,0.80,0.65,1.0
4,0.35,0.2,6.0,0.1,0.65,0.65,1.0


In [23]:
y.head()

0    7.275071e-104
1    2.403756e-111
2    4.701355e-110
3    8.296506e-112
4    3.113991e-109
Name: plateau_pop, dtype: float64

In [24]:
X_train, X_t, y_train, y_t = train_test_split(X, y, test_size=0.3)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
104332,0.75,0.6,7.0,1.0,0.80,0.50,100.0
33714,0.65,0.1,4.0,0.4,0.50,0.80,100.0
96435,0.75,0.7,2.0,0.6,0.95,0.95,100.0
25233,0.45,0.2,5.0,0.5,0.95,0.65,50.0
73706,0.65,0.2,10.0,0.6,0.80,0.50,1.0
...,...,...,...,...,...,...,...
289593,0.45,0.3,4.0,0.9,0.50,0.80,100.0
98878,0.25,0.1,3.0,0.2,0.95,0.95,100.0
3735,0.85,0.8,6.0,0.3,0.50,0.50,100.0
82879,0.85,0.5,4.0,0.3,0.95,0.95,100.0


In [25]:
X_test,X_val, y_test, y_val = train_test_split(X_t, y_t, test_size=0.4)
X_val

,b,d,death_r,dd,sd_b,sd_d,initial_pop
260087,0.95,0.8,7.0,0.9,0.80,0.95,1.0
246558,0.85,0.2,1.0,0.1,0.95,0.65,1.0
149695,0.85,0.3,10.0,0.3,0.50,0.65,100.0
26436,0.95,0.7,3.0,0.7,0.65,0.80,1.0
2446,0.85,0.6,8.0,0.5,0.95,0.65,50.0
...,...,...,...,...,...,...,...
125853,0.55,0.2,1.0,0.7,0.50,0.95,100.0
24361,0.75,0.4,4.0,0.8,0.95,0.50,100.0
170714,0.55,0.2,10.0,0.1,0.50,0.50,1.0
48681,0.75,0.2,4.0,0.1,0.65,0.95,100.0


In [26]:
print("train len", len(y_train))
print("test len ", len(y_t))
print("val len ", len(y_val))


train len 155197
test len  66514
val len  26606


In [27]:
# нормализация данных
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns = X_val.columns)
X_train

,b,d,death_r,dd,sd_b,sd_d,initial_pop
0,0.333681,1.078948,0.551968,1.565597,0.446191,-1.346035,1.252256
1,-0.104382,-1.185377,-0.459288,-0.521766,-1.344392,0.444994,1.252256
2,0.333681,1.531813,-1.133458,0.174022,1.341482,1.340509,1.252256
3,-0.980509,-0.732512,-0.122202,-0.173872,1.341482,-0.450520,0.022340
4,-0.104382,-0.732512,1.563223,0.174022,0.446191,-1.346035,-1.182978
...,...,...,...,...,...,...,...
155192,-0.980509,-0.279647,-0.459288,1.217703,-1.344392,0.444994,1.252256
155193,-1.856635,-1.185377,-0.796373,-1.217553,1.341482,1.340509,1.252256
155194,0.771744,1.984679,0.214883,-0.869659,-1.344392,-1.346035,1.252256
155195,0.771744,0.626083,-0.459288,-0.869659,1.341482,1.340509,1.252256


In [28]:
scaler_y = StandardScaler()
scaler_y.fit(y_train.to_frame())
y_train = scaler_y.transform(y_train.to_frame())
y_test = scaler_y.transform(y_test.to_frame())
y_val = scaler_y.transform(y_val.to_frame())
y_train

array([[-0.17838051],
       [-0.00985577],
       [-0.17838051],
       ...,
       [-0.17838051],
       [-0.17838051],
       [-0.17838051]])